In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
import json
import pandas as pd
import numpy as np
import nltk
import re
from collections import defaultdict

BiLSTM(Q)
BiLSTM(C)
softmax([Ocs, Oce]WOq)
cross_entropy

In [17]:


def cleanAnswer(a):
    clean_a = []
    for synonym_lst in a:
        lst = []
        for synonym in synonym_lst:
            lst.append(bioclean(synonym))
        clean_a.append(lst)
    return clean_a


    
def ansInSnippet(a, snippet):
    snippet = snippet.lower()
    for synonym_lst in a:
        for synonym in synonym_lst:
            if synonym.lower() in snippet:
                return True
    return False

def getQAPairClean(qa_pair):
    qa_pair_clean = []
    for (q, a, t, s) in qa_pair:
        s_clean = []
        a_clean = cleanAnswer(a)
        q = bioclean(q)
        for snippet in s:
            snippet = bioclean(snippet)
            s_clean.append(snippet)
        qa_pair_clean.append([q, a, t, s, s_clean, a_clean, []])
    return qa_pair_clean

def getQAPairTraining(qa_pair):
    qa_pair_clean = getQAPairClean(qa_pair)
    qa_pair_train = []
    p = []
    for (q, a, t, s, s_clean, a_clean, _) in qa_pair_clean:
        s_train = []
        s_ori = []
        for (i, snippet) in enumerate(s_clean):
            spans = getSpans(a_clean, snippet)
            if len(spans):
                p.append(spans)
                s_ori.append(s[i])
                s_train.append(snippet)
        if len(s_ori):
            qa_pair_train.append([q, a, t, s_ori, s_train, a_clean, p])
    return qa_pair_train

def text2List(qa_pair_clean):
    word_set = set()
    qa_pair_t2l = []
    for (q, a, t, s, s_clean, a_clean, p) in qa_pair_clean:
        s_clean_t2l = []
        for snippet in s_clean:
            word_seq = snippet.split()
            word_set.update(word_seq)
            s_clean_t2l.append(word_seq)
        q_t2l = q.split()
        word_set.update(q_t2l)
        for ans
        
        
    return (q)
    


def getCleanQAPair(qa_pair):
    qa_pair_clean = []
    for (q, a, t, s, p) in qa_pair:
        q = bioclean(q).split()
        a = [bioclean(x).split() for x in a]
        t = t
        s = [bioclean(x).split() for x in s]
        qa_pair_clean.append([q, a, t, s, p])
    return qa_pair_clean



def loadEmbedGloVe(w_dct, vec_path):
    word2ind = defaultdict(lambda:len(word2ind))
    word_vec = []

    with open(vec_path, encoding='utf-8') as f:
        for i, line in enumerate(f):
            line = line.strip('\n').split(' ')
            word = bioclean(line[0])
            if word in w_dct and word not in word2ind:
                vec = np.array([float(x) for x in line[1:]])
                word_vec.append(vec)
                word2ind[word]
    print (len(word2ind), 'words found for all', len(w_dct), 'words')

    return (np.array(word_vec), word2ind)


IndentationError: expected an indented block (<ipython-input-17-ab57ab3cdb8f>, line 39)

In [6]:
with open('./BioASQ-trainingDataset5b.txt', encoding='utf8') as f:
    data = json.load(f)
def getQAPair(data):
    qa_pair = []
    count = 0
    for obj in data['questions']:
        if obj['type'] in ['factoid', 'list']:
            q = obj['body']
            a = obj['exact_answer']
            if obj['type'] == 'factoid' and not isinstance(a[0], list):
                a = [a]
            t = obj['type']
            s = []
                        
            for snippet in obj['snippets']:
                snippet = snippet['text']
                if snippet not in s:
                    s.append(snippet)
                else:
                    count += 1
            if len(s) > 0:
                qa_pair.append([q, a, t, s])
    print ('duplicate snippet count:', count)
    return qa_pair

print ('getting qa pair from data')
qa_pair = getQAPair(data)
print (qa_pair[0])

def tokenize(qa_pair):
    from nltk.tokenize import word_tokenize
    new_qa_pair = []
    for (q, a, t, s) in qa_pair:
        new_q = word_tokenize(q)
        new_a = []
        for answer_lst in a:
            new_a.append([word_tokenize(answer) for answer in answer_lst])
        new_t = t
        new_s = [word_tokenize(snippet) for snippet in s]
        new_qa_pair.append([new_q, new_a, new_t, new_s])
    return new_qa_pair

print ('tokenizing qa pairs')
qa_pair = tokenize(qa_pair)
print (qa_pair[0])

def getSpans(a, snippet):
    snippet = ' '.join(snippet).lower()
    spans = []
    for synonym_lst in a:
        for synonym in synonym_lst:
            synonym = ' '.join(synonym).lower()
            synonym = re.escape(synonym)
            span = [(m.start(), m.end() - 1) for m in re.finditer(synonym, snippet)]
            
            span = [(len(snippet[:start].split(' ')) - 1, len(snippet[:end + 1].split(' ')) - 1) for (start, end) in span]
            if len(span):
                spans += span
    return spans

def addSpan(qa_pair):
    new_qa_pair = []
    for (q, a, t, s) in qa_pair:
        new_s, new_span_lst = [], []
        span_lst = [getSpans(a, snippet) for snippet in s]
        for (snippet, spans) in zip(s, span_lst):
            if len(spans):
                new_s.append(snippet)
                new_span_lst.append(spans)
        new_qa_pair.append([q, a, t, new_s, new_span_lst])
    return new_qa_pair

print ('adding spans into the qa_pair')
qa_pair = addSpan(qa_pair)
print (qa_pair[0])
print (qa_pair[0][1])
for (snippet, spans) in zip(qa_pair[0][3], qa_pair[0][4]):
    print (spans)
    print (snippet)
    for (start, end) in spans:
        print (snippet[start:end + 1])

def bioclean(t):
    return ' '.join(re.sub('[.,?;*!%^&_+():-\[\]{}]', '',
                        (t.replace('"', '').replace('/', '')
                         .replace('\\', '').replace("'", '')
                         .strip().lower()))
                    .split())

def getWordDict(qa_pair, cleanFunc):
    dct = set()
    for (q, a, t, s, span_lst) in qa_pair:
        for word in q:
            dct.add(cleanFunc(word))
        for ans_lst in a:
            for ans in ans_lst:
                for word in ans:
                    dct.add(cleanFunc(word))
        for snippet in s:
            for word in snippet:
                dct.add(cleanFunc(word))
    return dct

print ('getting bio word dictionary')
bio_wdict = getWordDict(qa_pair, bioclean)

def loadEmbedBioASQ(w_dct, vec_path, type_path):
    linenum2word = {}
    with open(type_path, encoding='utf8') as f:
        for i, line in enumerate(f):
            w = line.strip('\n')
            if w in w_dct:
                linenum2word[i] = w
    print (len(linenum2word), 'words found for all', len(w_dct), 'words')
    word_vec = []
    word2ind = defaultdict(lambda:len(word2ind))
    with open(vec_path) as f:
        for i, line in enumerate(f):
            if i in linenum2word:
                word = linenum2word[i]
                vec = np.array([float(x) for x in line.strip('\n').split()])
                word_vec.append(vec)
                word2ind[word]
    word2ind['<Unk>']
    norm1 = np.sum(word_vec, axis=0).mean()
    np.random.seed(42)
    word_vec.append(norm1*np.random.rand(len(word_vec[0])))
    return (np.array(word_vec), word2ind)

print ('loading embedding vector for words')
(word_vec_bio, word2ind_bio) = loadEmbedBioASQ(bio_wdict, vec_path='./vectors.txt', type_path='./types.txt')
ind2word_bio = dict((v,k) for k,v in word2ind_bio.items())

def word2Ind(word2ind, qa_pair, cleanFunc):
    new_qa_pair = []
    for (q, a, t, s, span_lst) in qa_pair:
        q_ind = [word2ind.get(cleanFunc(w), word2ind['<Unk>']) for w in q]
        s_ind = []
        for snippet in s:
            s_ind.append([word2ind.get(cleanFunc(w), word2ind['<Unk>']) for w in snippet])
        new_qa_pair.append([q_ind, s_ind, span_lst, t])
    return new_qa_pair

bio_qa_ind = word2Ind(word2ind_bio, qa_pair, bioclean)

def flattenSpanList(qa_pair):
    new_qa_pair = []
    for (q, s, span_lst, t) in qa_pair:
        for (snippet, spans) in zip(s, span_lst):
            for span in spans:
                new_qa_pair.append([q, snippet, span, t])
    return new_qa_pair

from sklearn.model_selection import train_test_split
train, test = train_test_split(bio_qa_ind, test_size=0.2, random_state=32)
train, validate = train_test_split(train, test_size=0.2, random_state=32)
print ('flatten span list')
train_flat = flattenSpanList(train)
test_flat = flattenSpanList(test)
validate_flat = flattenSpanList(validate)
bio_qa_ind_flat = flattenSpanList(bio_qa_ind)
print (bio_qa_ind_flat[0])

getting qa pair from data
duplicate snippet count: 5051
['List signaling molecules (ligands) that interact with the receptor EGFR?', [['epidermal growth factor'], ['betacellulin'], ['epiregulin'], ['heparin-binding epidermal growth factor'], ['transforming growth factor-α'], ['amphiregulin'], ['epigen']], 'list', ['the epidermal growth factor receptor (EGFR) ligands, such as epidermal growth factor (EGF) and amphiregulin (AREG)', ' EGFR ligands epidermal growth factor (EGF), amphiregulin (AREG) and transforming growth factor alpha (TGFα)', ' EGFR and its ligand EGF ', 'Among EGFR ligands, heparin-binding EGF-like growth factor, TGF-α and Betacellulin (BTC) are produced in the tumor microenvironment of FDC-S at RNA level. ', '. Plasma amphiregulin (AR), epidermal growth factor (EGF), transforming growth factor-α, and heparin binding-EGF were assessed by ELISA in 45 chemorefractory mCRC patients', 'Among EGFR ligands, heparin-binding epidermal growth factor (HB-EGF)', ' Of the six known 

['Among', 'EGFR', 'ligands', ',', 'heparin-binding', 'epidermal', 'growth', 'factor', '(', 'HB-EGF', ')']
['epidermal', 'growth', 'factor']
['heparin-binding', 'epidermal', 'growth', 'factor']
[(8, 8), (10, 10), (15, 17), (22, 22), (25, 25)]
['the', '7', 'known', 'EGFR', 'ligands', '(', 'EGF', ',', 'betacellulin', ',', 'epiregulin', ',', 'heparin-binding', 'EGF', ',', 'transforming', 'growth', 'factor-α', '[', 'TGF-α', ']', ',', 'amphiregulin', ',', 'and', 'epigen', ')']
['betacellulin']
['epiregulin']
['transforming', 'growth', 'factor-α']
['amphiregulin']
['epigen']
[(15, 17)]
['In', 'this', 'article', ',', 'however', ',', 'we', 'demonstrate', 'that', 'PEPD', 'directly', 'binds', 'to', 'and', 'activates', 'epidermal', 'growth', 'factor', 'receptor', '(', 'EGFR', ')', ',']
['epidermal', 'growth', 'factor']
[(0, 2)]
['Epidermal', 'growth', 'factor', '(', 'EGF', ')', 'family', 'peptides', 'are', 'ligands', 'for', 'the', 'EGF', 'receptor', '(', 'EGFR', ')', '.']
['Epidermal', 'growth', '

In [7]:
np.save('word2ind_bio.npy', dict(word2ind_bio))
np.save('ind2word_bio.npy', dict(ind2word_bio))
np.save('word_vec_bio.npy', word_vec_bio)

In [8]:
train_flat = flattenSpanList(train)
test_flat = flattenSpanList(test)
validate_flat = flattenSpanList(validate)
bio_qa_ind_flat = flattenSpanList(bio_qa_ind)

In [80]:
word_vec_bio.shape

(16456, 200)

In [ ]:
# extract question-answer-context pair
# lower, tokenization, find span
# question-answer-context -> all to list
# get word dict in bioclean, punctuation dict, assign an embedding to each punctuation
# translate the list token to number including unknown

# train, validation, test, set split
# get question-answer-context-span_list to question-answer-context-span

# evaluation 

In [38]:
(word_vec_bio, word2ind_bio) = loadEmbedBioASQ(w_dct, vec_path='./vectors.txt', type_path='./types.txt')

15358 words found for all 16626 words


In [49]:
(word_vec_GloVe, word2ind_GloVe) = loadEmbedGloVe(w_dct, './glove.6B.300d.txt')

10540 words found for all 16626 words


In [41]:
len(word_vec[0])

200

In [43]:
word_vec.shape

(15358, 200)

In [27]:
len(w_dct)

16626

In [38]:
snippet_count.describe()

count    899.000000
mean      14.303671
std       12.157930
min        0.000000
25%        6.000000
50%       11.000000
75%       19.000000
max      121.000000
dtype: float64

In [39]:
len(qa_pair)

486

In [42]:
data['questions'][0]

{'body': 'Is Hirschsprung disease a mendelian or a multifactorial disorder?',
 'concepts': ['http://www.disease-ontology.org/api/metadata/DOID:10487',
  'http://www.nlm.nih.gov/cgi/mesh/2015/MB_cgi?field=uid&exact=Find+Exact+Term&term=D006627',
  'http://www.disease-ontology.org/api/metadata/DOID:11372',
  'http://www.nlm.nih.gov/cgi/mesh/2015/MB_cgi?field=uid&exact=Find+Exact+Term&term=D020412'],
 'documents': ['http://www.ncbi.nlm.nih.gov/pubmed/15829955',
  'http://www.ncbi.nlm.nih.gov/pubmed/15617541',
  'http://www.ncbi.nlm.nih.gov/pubmed/12239580',
  'http://www.ncbi.nlm.nih.gov/pubmed/8896569',
  'http://www.ncbi.nlm.nih.gov/pubmed/6650562',
  'http://www.ncbi.nlm.nih.gov/pubmed/20598273',
  'http://www.ncbi.nlm.nih.gov/pubmed/21995290',
  'http://www.ncbi.nlm.nih.gov/pubmed/23001136'],
 'id': '55031181e9bde69634000014',
 'ideal_answer': ["Coding sequence mutations in RET, GDNF, EDNRB, EDN3, and SOX10 are involved in the development of Hirschsprung disease. The majority of these

In [ ]:
#preprocessing
with open('./types.txt') as ftype:
    for line in ftype:
        line = line.strip('\n')

In [27]:
from torch.utils.data import DataLoader
import time
torch.manual_seed(1)

train_dataloader = DataLoader(train_flat, shuffle=True, batch_size=1)
validate_dataloader = DataLoader(validate_flat, shuffle=True)
model = BaselineModel(hidden_dim=200, embeddings=word_vec_bio)
model.cuda()
loss = torch.nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for epoch in range(10):
    train_loss = 0
    count = 0
    start = time.time()
    for (q, snippet, span, t) in train_flat:
        count += 1
        print (count)
        if count % 100 == 0:
            print (count, 'th cases')
            print (time.time() - start)
            
            validate_loss = 0
            for (q, snippet, span, t) in validate_flat[:100]:
                y = model(q, snippet)
                loss_val = loss(y, autograd.Variable(torch.cuda.LongTensor([span[0]*len(snippet) + span[1]]))).data[0]
            validate_loss += loss_val
            print ('validation loss:', validate_loss)
        
        optimizer.zero_grad()
        y = model(q, snippet)
        loss_val = loss(y, autograd.Variable(torch.cuda.LongTensor([span[0]*len(snippet) + span[1]])))
        loss_val.backward()
        optimizer.step()
        train_loss += loss_val.data[0]
        
    print ('epoch:', epoch, 'train loss:', train_loss)
    


1
2


RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

In [17]:
loss_val.data.cpu().numpy()

array([-0.00093584], dtype=float32)

## model

In [20]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)

class BaselineModel(nn.Module):
    def __init__(self, hidden_dim, embeddings, batch_size=1, num_layers=1):
        super(BaselineModel, self).__init__()
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim

        vocab_size = embeddings.shape[0]
        embedding_dim = embeddings.shape[1]
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.word_embeddings.weight = nn.Parameter(torch.FloatTensor(embeddings))
        
        self.qlstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)
        self.clstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)

        # The linear layer that maps from hidden state space to tag space
        self.qhidden = self.initHidden()
        self.chidden = self.initHidden()
        
        self.out2end = nn.Linear(2*hidden_dim*2, 2*hidden_dim*2)
        self.out2start = nn.Linear(2*hidden_dim*2, 1)

        
    def attnCombine(self, qlstm_out, clstm_out):
        # (batch_size, hidden_size*num_directions, qseq_len)
        qlstm_out_tmp = qlstm_out.permute(1, 2, 0)
        # (batch_size, cseq_len, hidden_size*num_directions)
        clstm_out_tmp = clstm_out.permute(1, 0, 2)
        # (batch_size, cseq_len, qseq_len)
        attn_weights = F.softmax(torch.bmm(clstm_out_tmp, qlstm_out_tmp), dim=2)
        # (batch_size, qseq_len, hidden_size*num_directions)
        qlstm_out_tmp2 = qlstm_out.permute(1, 0, 2)
        # (batch_size, cseq_len, hidden_size*num_directions)
        attn_vec = torch.bmm(attn_weights, qlstm_out_tmp2)
        clstm_attn_out = torch.cat((clstm_out_tmp, attn_vec), dim=2)
        # (batch_size, cseq_len, 2*hidden_size*num_directions)
        return clstm_attn_out
    
    def initHidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers*direction_num, minibatch_size, hidden_dim)
        batch_size = self.batch_size
        num_layers = self.num_layers
        return (autograd.Variable(torch.zeros(num_layers*2, batch_size, self.hidden_dim).cuda()),
                autograd.Variable(torch.zeros(num_layers*2, batch_size, self.hidden_dim).cuda()))

    def forward(self, question, context):
        qembeds = self.word_embeddings(autograd.Variable(torch.cuda.LongTensor([question])))
        cembeds = self.word_embeddings(autograd.Variable(torch.cuda.LongTensor([context])))
        # output : (seq_len, batch_size, hidden_size*num_directions)
        qlstm_out, self.qhidden = self.qlstm(
            qembeds.view(len(question), 1, -1), self.qhidden)
        clstm_out, self.chidden = self.clstm(
            cembeds.view(len(context), 1, -1), self.chidden)
#         # (batch_size, cseq_len, 2*hidden_size*num_directions)
#         clstm_attn_out = self.attnCombine(qlstm_out, clstm_out)

        # (batch_size, hidden_size*num_directions, qseq_len)
        qlstm_out_tmp = qlstm_out.permute(1, 2, 0)
        # (batch_size, cseq_len, hidden_size*num_directions)
        clstm_out_tmp = clstm_out.permute(1, 0, 2)
        # (batch_size, cseq_len, qseq_len)
        attn_weights = F.softmax(torch.bmm(clstm_out_tmp, qlstm_out_tmp), dim=2)
        # (batch_size, qseq_len, hidden_size*num_directions)
        qlstm_out_tmp2 = qlstm_out.permute(1, 0, 2)
        # (batch_size, cseq_len, hidden_size*num_directions)
        attn_vec = torch.bmm(attn_weights, qlstm_out_tmp2)
        clstm_attn_out = torch.cat((clstm_out_tmp, attn_vec), dim=2)
        # (batch_size, cseq_len, 2*hidden_size*num_directions)

        # (batch_size, cseq_len, 2*hidden_size*num_directions)
        end_space = self.out2end(clstm_attn_out)
        # (batch_size, 2*hidden_size*num_directions, cseq_len)
        clstm_attn_out_tmp = clstm_attn_out.permute(0, 2, 1)
        
        # (batch_size, cseq_len, 1)
        start_prob = F.sigmoid(self.out2start(clstm_attn_out))
        # (batch_size, cseq_len, cseq_len)
        end_prob = F.softmax(torch.bmm(end_space, clstm_attn_out_tmp), dim=2)
        
        # (batch_size, cseq_len, cseq_len)
        span_prob = end_prob*start_prob
        return F.normalize(span_prob.view(self.batch_size, -1), dim=1, p=1)

In [7]:
mini = None
maxi = None
with open('./vectors.txt') as f:
    for i, line in enumerate(f):
        vec = np.array([float(x) for x in line.strip('\n').split()])
        mini = min(vec) if mini is None else min(mini, min(vec))
        maxi = max(vec) if maxi is None else max(maxi, max(vec))

In [8]:
print (mini, maxi)
# CAVEAT: problems in unknown token initialization

-2.25082 3.10778


In [131]:
torch.LongTensor([[[3], [2], [99]], [[4], [1], [88]]]).view(2, -1)


  3   2  99
  4   1  88
[torch.LongTensor of size 2x3]

In [139]:
F.normalize(torch.FloatTensor([[1, 2], [33, 2]]), dim=1, p=1)


 0.3333  0.6667
 0.9429  0.0571
[torch.FloatTensor of size 2x2]

In [169]:
torch.LongTensor(1, 8, 8).random_(0, 25)


(0 ,.,.) = 
   0   6  15  22  14  13  11  22
  18  19   0   1  19  17  12  20
  11  20  11   5  23  24   9   4
   4  15   0  18   2   8   8  10
   0  14   5   9  16  19   4   8
  13  12  15   3  16  18  21   8
  10  19   6   0  18  12  12  14
  13  10  14   2  12  21  14  22
[torch.LongTensor of size 1x8x8]